Fetching & Converting table from a given link to a pandas Dataframe

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

data = r.text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
table_string = soup.table.prettify()

In [5]:
table_list = pd.read_html(table_string)
table_list

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                           Neighborhood  
 0                                                  NaN  
 1                                                  NaN  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                                NaN  
 176                                                NaN  
 177                

In [6]:
df_table = table_list[0]
df_table

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [7]:
df_table.drop(df_table.loc[df_table['Borough']=='Not assigned'].index, inplace=True)

In [8]:
df_table.index = [x for x in range(df_table.shape[0])]
df_table

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
str(df_table.shape[0])+' rows'

'103 rows'

#######################################################################

Adding the columns of Latitude and Longitude

##################################################################

In [11]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')

In [12]:
coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [55]:
#coordinates.loc[coordinates['Postal Code'] == 'M1B', 'Longitude'][0]
latitudes = []
longitudes = []
for code in df_table['Postal Code']:
   lat = coordinates.loc[coordinates['Postal Code'] == code, 'Latitude']
   latitudes.append(lat[lat.index[0]])
   lng = coordinates.loc[coordinates['Postal Code'] == code, 'Longitude']
   longitudes.append(lng[lng.index[0]]) 

In [57]:
df_table['Latitude'] = latitudes

In [58]:
df_table['Longitude'] = longitudes

In [61]:
df_table

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [64]:
str(df_table.shape[0])+' rows'

'103 rows'

########################################################################

Exploring & Clustering the neighborhoods in Toronto

########################################################################

In [66]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_table['Borough'].unique()),
        df_table.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [68]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Map of Toronto with neighborhoods superimposed on top.

In [69]:
import folium
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_table['Latitude'], df_table['Longitude'], df_table['Borough'], df_table['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [70]:
CLIENT_ID = 'I11VAB3HANLKQFXW25SCZ1N5YQEJKTOIDCLGT1VNB4GKN2NQ' # your Foursquare ID
CLIENT_SECRET = 'MX034V5THSEG24PHY5TZAIFWZHD2I1TN3XZJAK5KX1LSOFTZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I11VAB3HANLKQFXW25SCZ1N5YQEJKTOIDCLGT1VNB4GKN2NQ
CLIENT_SECRET:MX034V5THSEG24PHY5TZAIFWZHD2I1TN3XZJAK5KX1LSOFTZ


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [71]:
df_table.loc[0, 'Neighborhood']

'Parkwoods'

Get the neighborhood's latitude and longitude values.

In [72]:
neighborhood_latitude = df_table.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_table.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_table.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


#### Now, let's get the top 100 venues that are in Parkwoods within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [73]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=I11VAB3HANLKQFXW25SCZ1N5YQEJKTOIDCLGT1VNB4GKN2NQ&client_secret=MX034V5THSEG24PHY5TZAIFWZHD2I1TN3XZJAK5KX1LSOFTZ&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

Send the GET request and examine the resutls

In [75]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec93062fb34b5001b3d5b44'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [76]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [77]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [79]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

In [80]:
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4e8d9dcdd5fbbbb6b3003c7b-0,0,"[{'summary': 'This spot is popular', 'type': '...",4e8d9dcdd5fbbbb6b3003c7b,Brookbanks Park,Toronto,43.751976,-79.332140,"[{'label': 'display', 'lat': 43.75197604605557...",245,CA,Toronto,ON,Canada,"[Toronto, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[]
1,e-0-4cb11e2075ebb60cd1c4caad-1,0,"[{'summary': 'This spot is popular', 'type': '...",4cb11e2075ebb60cd1c4caad,Variety Store,29 Valley Woods Road,43.751974,-79.333114,"[{'label': 'display', 'lat': 43.75197441585782...",312,CA,Toronto,ON,Canada,"[29 Valley Woods Road, Toronto ON, Canada]","[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",0,[]


In [81]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [82]:
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",43.751976,-79.332140
1,Variety Store,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",43.751974,-79.333114


In [83]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

In [84]:
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [85]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [86]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


We just explored the nearby top venues of only one neighborhood i.e. Parkwoods. we can do the same for all the neighborhoods as shown:

In [94]:
count = 0
toronto_venues = []
for i in range(df_table.shape[0]):
    neighborhood_latitude = df_table.loc[i, 'Latitude'] # neighborhood latitude value
    neighborhood_longitude = df_table.loc[i, 'Longitude'] # neighborhood longitude value
    neighborhood_name = df_table.loc[i, 'Neighborhood'] # neighborhood name
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
    
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    nearby_venues = pd.json_normalize(venues)
    try:
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby_venues =nearby_venues.loc[:, filtered_columns]
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        # print(nearby_venues.head(1))   //for printing venues
        count+=1
        print('{} - {} venues were returned by Foursquare for neighborhood {}.'.format(count,nearby_venues.shape[0],neighborhood_name))
    except:
        pass

1 - 2 venues were returned by Foursquare for neighborhood Parkwoods.
2 - 5 venues were returned by Foursquare for neighborhood Victoria Village.
3 - 46 venues were returned by Foursquare for neighborhood Regent Park, Harbourfront.
4 - 13 venues were returned by Foursquare for neighborhood Lawrence Manor, Lawrence Heights.
5 - 36 venues were returned by Foursquare for neighborhood Queen's Park, Ontario Provincial Government.
6 - 2 venues were returned by Foursquare for neighborhood Malvern, Rouge.
7 - 5 venues were returned by Foursquare for neighborhood Don Mills.
8 - 10 venues were returned by Foursquare for neighborhood Parkview Hill, Woodbine Gardens.
9 - 100 venues were returned by Foursquare for neighborhood Garden District, Ryerson.
10 - 4 venues were returned by Foursquare for neighborhood Glencairn.
11 - 2 venues were returned by Foursquare for neighborhood Rouge Hill, Port Union, Highland Creek.
12 - 22 venues were returned by Foursquare for neighborhood Don Mills.
13 - 9 venu